In [32]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Reshape,Input,Dense,Flatten, BatchNormalization
from keras.models import Model,Sequential
from keras.optimizers import Adam
from keras.layers import LeakyReLU

In [29]:
from keras.datasets import mnist

In [43]:
import gzip
import sys
import pickle as cPickle
f = gzip.open('/Users/s5284664/Downloads/mnist.pkl.gz', 'rb')
if sys.version_info < (3,):
    data = cPickle.load(f)
else:
    data = cPickle.load(f, encoding='bytes')
f.close()


In [44]:
img_row=28
img_cols=28
channel=1
img_shape=(img_row,img_cols,channel)

In [45]:
#Dense layers are used for generator
def generator1():
    noise_shape=(100,)
    model=Sequential()
    model.add(Dense(256,input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape),activation='tanh'))
    model.add(Reshape(img_shape))
    model.summary()
    noise=Input(shape=noise_shape)
    img=model(noise)
    return Model(noise,img)

In [46]:
def discriminator():
    model=Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1,activation='sigmoid'))
    model.summary()
    img=Input(shape=img_shape)
    validity=model(img)
    return Model(img,validity)
   

In [55]:
def train(epochs,batch_size=128,save_interval=50):
    (x_train,_),(_,_)=data
    x_train=(x_train.astype(np.float32)-127.5)/127.5
    x_train=np.expand_dims(x_train,axis=3)
    half_batch=int(batch_size/2)
    for epoch in range(epochs):
        idx=np.random.randint(0,x_train.shape[0],half_batch)
        imgs=x_train[idx]
        
        #training descriminator
        noise=np.random.normal(0,1,(half_batch,100))
        gen_imgs=generator.predict(noise)
        d_loss_real=descriminator.train_on_batch(gen_imgs,np.ones(half_batch,1))
        d_loss_fake=descriminator.train_on_batch(gen_imgs,np.zeros(half_batch,1))
        d_loss=0.5*np.add(d-loss_real,d_loss_fake)
        
        
        #training generator
        noise=np.random.normal(0,1,(half_batch,100))
        valid_y=np.array([1]*batch_size)
        g_loss=combined.train_on_batch(noise,valid_y)
        print("%d[D loss:%f,acc.:%.2f%%][G loss:%f]"%(epoch,d_loss[0],100*d_loss[1],g_loss[0]))
        if epoch% save_interval==0:
            save_imgs(epoch)

In [56]:
def save_imgs(epoch):
    r,c=5,5
    noise=np.random.normal(0,1,(r*c,100))
    gen_imgs=generator.predict(noise)
    gen_imgs=0.5*gen_imgs+0.5
    fig,axs=plt.subplot(r,c)
    cnt=0
    for i in range(r):
        for i in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0],cmap='gray')
            axs[i,j].axis('off')
            cnt+=1
    fig.savefig("images/mnist_%d.png"% epoch)
    plt.close()

In [57]:
optimizer=Adam(0.0002,0.5)
descriminator=discriminator()
descriminator.compile(loss="binary_crossentropy", optimizer=optimizer,metrics=['accuracy'])

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 dense_28 (Dense)            (None, 512)               401920    
                                                                 
 leaky_re_lu_20 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_29 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_21 (LeakyReLU)  (None, 256)               0         
                                                                 
 dense_30 (Dense)            (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-tr

In [58]:
generator=generator1()
generator.compile(loss="binary_crossentropy", optimizer=optimizer)
z=Input(shape=(100,))
img=generator(z)
descriminator.trainable=False
valid=descriminator(img)
combined=Model(z,valid)
combined.compile(loss='binary_crossentropy',optimizer=optimizer)
train(epochs=10,batch_size=12,save_interval=10)
generator.save('generator_model_test.h5')

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 256)               25856     
                                                                 
 leaky_re_lu_22 (LeakyReLU)  (None, 256)               0         
                                                                 
 batch_normalization_12 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 dense_32 (Dense)            (None, 512)               131584    
                                                                 
 leaky_re_lu_23 (LeakyReLU)  (None, 512)               0         
                                                                 
 batch_normalization_13 (Bat  (None, 512)              2048      
 chNormalization)                                     

TypeError: Cannot interpret '1' as a data type